In [280]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time

# headers for spoof
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
init_url = 'https://www.list-org.com'

In [263]:
# Helper functions
def p_splitter(req_tag):
    result = {}
    for p in req_tag.findAll('p'):
        req_string = p.text.split(':')
        result[req_string[0]]=req_string[1].strip()
    return result 

def get_owners(owner_tag):
    structure = []
    titles = [title.text for title in owner_tag.table.tr.findAll('td')]
    for owner in owner_tag.table.findAll('tr')[1:]:
        owner_val = {titles[i]:val.text for i,val in enumerate(owner.findAll('td'))}
        structure.append(str(owner_val))
    return structure

def get_finance(finance_tag):
    financial_data = []
    all_tags = finance_tag.table.findAll('tr')[1:]
    titles = [title.text for title in all_tags[0].findAll('td')]
    for financial in all_tags[1:-1]:
        financial_val = {titles[i]:val.text for i,val in enumerate(financial.findAll('td'))}
        financial_data.append(str(financial_val))
    return financial_data

In [7]:
a = requests.get('https://www.list-org.com/search?type=all&work=on&sort=&page=2', headers=headers)

In [17]:
# Finding the urls of active companies on the webpage
soup = BeautifulSoup(a.text)
req_urls = soup.find("div", {"class": "org_list"}).findAll("a")

In [23]:
# Loading company page
c = requests.get(init_url+req_urls[0]['href'],headers=headers)

In [37]:
page_soup = BeautifulSoup(c.text).find("div", {"class": "content"})

In [65]:
# Skipping display:none tag
general_info = page_soup.findAll("div", {"class": "card w-100 p-1 p-lg-3 mt-1"}, recursive=False)[1].table

In [265]:
company_dict = {}

for tag in general_info.findAll('tr'):
    # Filter bad tags
    if tag.a!=None:
        company_dict[tag.i.text[:-1]] = tag.a.text
    else:
        company_dict[tag.i.text[:-1]] = tag.findAll('td')[1].text

In [266]:
# Loop over cards
for card in page_soup.findAll("div", {"class": "card w-100 p-1 p-lg-3 mt-2"}, recursive=False):
    if card.h6.text=='Контактная информация:':
        contact_info = p_splitter(card)
    elif card.h6.text=='Реквизиты компании:':
        requisites = p_splitter(card)
    elif card.h6.text=='Учредители:':
        # List for json
        owner_info = get_owners(card)
    elif card.h6.text.startswith('Результаты работы за'):
        # List for json
        last_year_finance_info = get_finance(card)

In [269]:
# Joining everything together

company_dict.update(contact_info)
company_dict.update(requisites)
company_dict['owners_structure'] = owner_info
company_dict['last_year_balance_sheet'] = last_year_finance_info

In [271]:
# Example result
company_dict

{'Полное юридическое наименование': 'ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО  "ФАРМСТАНДАРТ-ЛЕКСРЕДСТВА"',
 'Руководитель': 'Крейман Владимир Адольфович',
 'ИНН / КПП': '4631002737 / 463201001',
 'Уставной капитал': '22.0419 млн.руб.',
 'Численность персонала': '1813',
 'Количество учредителей': '1',
 'Дата регистрации': '18.10.2002',
 'Статус': 'Действующее',
 'Индекс': '305909',
 'Адрес': 'Г КУРСК,22,УЛ 2-Я АГРЕГАТНАЯ, 1 А/18',
 'GPS координаты': '51.736690521,36.245475769',
 'Юридический адрес': '305022, КУРСКАЯ ОБЛАСТЬ, Г. КУРСК, УЛ. АГРЕГАТНАЯ 2-Я, Д.1А/18',
 'Телефон': '+7 (4712) 34-12-80, +7 (4712) 26-14-65, 6-14-65',
 'Факс': '+7 (4712) 34-03-13, +7 (4712) 26-14-65',
 'E-mail': 'ozolotukhina@icnpharm.com',
 'Сайт': '',
 'ИНН': '4631002737',
 'КПП': '463201001',
 'ОКПО': '00155547',
 'ОГРН': '1024600945478',
 'ОКФС': '16 - Частная собственность',
 'ОКОГУ': '4210008 - Приватизированные предприятия',
 'ОКОПФ': '12247 - Публичные акционерные общества',
 'ОКТМО': '38701000001',
 'ФСФР': '4000

In [284]:
def get_company(company_url):
    # Loading company page
    c = requests.get(company_url,headers=headers)
    page_soup = BeautifulSoup(c.text).find("div", {"class": "content"})
    
    list_org_id = company_url.split('/')[-1]
    
    # Skipping display:none tag
    general_info = page_soup.findAll("div", {"class": "card w-100 p-1 p-lg-3 mt-1"}, recursive=False)[1].table
    company_dict = {'list_org_id':list_org_id}

    for tag in general_info.findAll('tr'):
        # Filter bad tags
        if tag.a!=None:
            company_dict[tag.i.text[:-1]] = tag.a.text
        else:
            company_dict[tag.i.text[:-1]] = tag.findAll('td')[1].text
    
    # If empty
    owner_info = '[]'
    last_year_finance_info = '[]'
    
    # Loop over cards
    for card in page_soup.findAll("div", {"class": "card w-100 p-1 p-lg-3 mt-2"}, recursive=False):
        if card.h6.text=='Контактная информация:':
            contact_info = p_splitter(card)
        elif card.h6.text=='Реквизиты компании:':
            requisites = p_splitter(card)
        elif card.h6.text=='Учредители:':
            # List for json
            owner_info = get_owners(card)
        elif card.h6.text.startswith('Результаты работы за'):
            # List for json
            last_year_finance_info = get_finance(card)
            
    # Joining everything together

    company_dict.update(contact_info)
    company_dict.update(requisites)
    company_dict['owners_structure'] = owner_info
    company_dict['last_year_balance_sheet'] = last_year_finance_info
    return company_dict

In [298]:
companies = []
# total claimed companies /50 round(4149878/50)
for page_n in range(1,5):
    page = requests.get(f'https://www.list-org.com/search?type=all&work=on&sort=&page={page_n}', headers=headers)
    # Finding the urls of active companies on the webpage
    soup = BeautifulSoup(page.text)
    req_urls = soup.find("div", {"class": "org_list"}).findAll("a")
    for url in req_urls: 
        companies.append(get_company(init_url+url['href']))
        time.sleep(1)
    print(f'Finished page {page_n}')

AttributeError: 'NoneType' object has no attribute 'findAll'

In [294]:
pd.DataFrame(companies).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Полное юридическое наименование  50 non-null     object
 1   Руководитель                     50 non-null     object
 2   ИНН / КПП                        48 non-null     object
 3   Уставной капитал                 48 non-null     object
 4   Численность персонала            31 non-null     object
 5   Количество учредителей           46 non-null     object
 6   Дата регистрации                 48 non-null     object
 7   Статус                           48 non-null     object
 8   Индекс                           50 non-null     object
 9   Адрес                            50 non-null     object
 10  GPS координаты                   41 non-null     object
 11  Юридический адрес                50 non-null     object
 12  Телефон                          50 no